In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext(master='local', appName = 'College App')

In [2]:
import findspark
findspark.init()
import pyspark
findspark.find()

'/Users/vunguyen/server/spark-2.4.6-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
'''
conf= pyspark.SparkConf().setAppName('tets_app').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
sc
'''

#sc= SparkContext()
spark = (SparkSession
         .builder
         .appName('College App')
         .getOrCreate())
spark

In [4]:
df = spark.read.csv('File/College.csv', header = True, inferSchema=True)

In [5]:
df.count()

777

In [6]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [7]:
df.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import round

In [9]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [10]:
assembler = VectorAssembler(
            inputCols=[ 'Apps',
                         'Accept',
                         'Enroll',
                         'Top10perc',
                         'Top25perc',
                         'F_Undergrad',
                         'P_Undergrad',
                         'Outstate',
                         'Room_Board',
                         'Books',
                         'Personal',
                         'PhD',
                         'Terminal',
                         'S_F_Ratio',
                         'perc_alumni',
                         'Expend',
                         'Grad_Rate'],
            outputCol='features')

In [11]:
output = assembler.transform(df)

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol = 'Private', outputCol='PrivateIndex')
output_fixed = indexer.fit(output).transform(output)

In [14]:
final_data = output_fixed.select('features','PrivateIndex')

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

In [17]:
dtc = DecisionTreeClassifier(featuresCol = 'features',labelCol = 'PrivateIndex')
rfc = RandomForestClassifier(featuresCol = 'features',labelCol = 'PrivateIndex')
gbt = GBTClassifier(featuresCol = 'features',labelCol = 'PrivateIndex')

In [18]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [19]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol = 'PrivateIndex',
                                                 predictionCol = 'prediction',
                                                 metricName='accuracy')

In [22]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [23]:
print('Decision Tree: {0:2.2f}%'.format(dtc_acc * 100))
print('Random Forest: {0:2.2f}%'.format(rfc_acc * 100))
print('Gradient Boost: {0:2.2f}%'.format(gbt_acc * 100))

Decision Tree: 91.16%
Random Forest: 93.02%
Gradient Boost: 91.63%
